In [1]:
import numpy as np
#from scipy.misc import imread, imresize
from sklearn import svm
import pickle

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = torch.device('cuda')

Variable setting! 

In [3]:
imtype=1; #images: 1=merge, 2=array
cat=19 #0-19 for which object category to attend to and readout for
layer = 0 #0-12 for which convolutional layer to apply attention at (if >12, will apply at all layers at 1/10th strength)
appwith = 'TCs' #what values to apply attention according to: 'TCs' or 'GRADs'
astrgs=np.arange(0,1.,.5) #attention strengths (betas)
TCpath='/scratch/bs4283/Data/Data/object_GradsTCs'  #folder with tuning curve and gradient files 
weight_path = '/scratch/bs4283/Data/Data' #folder with network and classifier weights
impath='/scratch/bs4283/Data/Data/objims/images' #folder where image files are kept
save_path = '/scratch/bs4283/Data/Data/savefi' #where to save the recording and performance files
Ncats = 20 #change to 5 if only using the categories available in the dryad files
rec_activity = True #record and save activity or no
binary_weight = '/scratch/bs4283/Data/Data/catbins/catbins'

imperim=5
traintype=3; #shouldnt be changed
bsize=15*imperim; # total number of images in each (true pos and true neg) class
impercat=np.floor(bsize/(Ncats-1)*1.); leftov=bsize%(Ncats-1)
bd=1; #bidirect or pos only (0)
attype=1 #1=mult, 2=add, 

if attype==1:  
    astrgs=astrgs
elif attype==2:  
    lyrBL=[20,100,150,150,240,240,150,150,80,20,20,10,1]

In [11]:
descat=cat 
descatpics=np.load(impath+'/merg5_c'+str(descat)+'.npz')['arr_0']

In [12]:
reader = tf.compat.v1.train.NewCheckpointReader('/scratch/bs4283/Data/Data/catbins/catbins/catbin_'+str(cat)+'.ckpt')

set gradient or tuning value

In [15]:
def make_gamats(oind,svec,attnmats=[]):
    with open(TCpath+'/CATgradsDetectTrainTCs_im'+str(imtype)+'.txt', "rb") as fp:  b = pickle.load(fp,encoding='latin1')
    for li in range(2):
        fv = b[li]
        fmvals=np.squeeze(fv[oind,:])/np.amax(np.abs(fv),axis=0)
        aval=np.expand_dims(np.expand_dims(fmvals,axis=0),axis=0)
        aval[aval==np.inf]=0
        aval[aval==-np.inf]=0; aval=np.nan_to_num(aval)
        if bd ==0:
            aval[aval<0]=0
        if attype == 1 :
            amat=np.ones((224,224,64))+np.tile(aval,[224,224,1])*svec[li]; amat[amat<0]=0
        elif attype == 2 :
            amat=np.tile(aval,[224,224,1])*svec[li]*lyrBL[li]
        attnmats.append(amat) 
    for li in range(2,4):
        fv = b[li]
        fmvals=np.squeeze(fv[oind,:])/np.amax(np.abs(fv),axis=0)
        aval=np.expand_dims(np.expand_dims(fmvals,axis=0),axis=0)
        aval[aval==np.inf]=0
        aval[aval==-np.inf]=0; aval=np.nan_to_num(aval)
        if bd ==0:
            aval[aval<0]=0
        if attype == 1 :
            amat=np.ones((112,112,128))+np.tile(aval,[112,112,1])*svec[li]; amat[amat<0]=0
        elif attype == 2 :
            amat=np.tile(aval,[112,112,1])*svec[li]*lyrBL[li]
        attnmats.append(amat)
    for li in range(4,7):
        fv = b[li]
        fmvals=np.squeeze(fv[oind,:])/np.amax(np.abs(fv),axis=0)
        aval=np.expand_dims(np.expand_dims(fmvals,axis=0),axis=0)
        aval[aval==np.inf]=0
        aval[aval==-np.inf]=0; aval=np.nan_to_num(aval)
        if bd ==0:
            aval[aval<0]=0
        if attype == 1 :
            amat=np.ones((56,56,256))+np.tile(aval,[56,56,1])*svec[li]; amat[amat<0]=0
        elif attype == 2 :
            amat=np.tile(aval,[56,56,1])*svec[li]*lyrBL[li]
        attnmats.append(amat) 
    for li in range(7,10):
        fv = b[li]
        fmvals=np.squeeze(fv[oind,:])/np.amax(np.abs(fv),axis=0)
        aval=np.expand_dims(np.expand_dims(fmvals,axis=0),axis=0)
        aval[aval==np.inf]=0
        aval[aval==-np.inf]=0; aval=np.nan_to_num(aval)
        if bd ==0:
            aval[aval<0]=0
        if attype == 1 :
            amat=np.ones((28,28,512))+np.tile(aval,[28,28,1])*svec[li]; amat[amat<0]=0
        elif attype == 2 :
            amat=np.tile(aval,[28,28,1])*svec[li]*lyrBL[li]
        attnmats.append(amat) 
    for li in range(10,13):
        fv = b[li]
        fmvals=np.squeeze(fv[oind,:])/np.amax(np.abs(fv),axis=0)
        aval=np.expand_dims(np.expand_dims(fmvals,axis=0),axis=0)
        aval[aval==np.inf]=0
        aval[aval==-np.inf]=0; aval=np.nan_to_num(aval)
        if bd ==0:
            aval[aval<0]=0
        if attype == 1 :
            amat=np.ones((14,14,512))+np.tile(aval,[14,14,1])*svec[li]; amat[amat<0]=0
        elif attype == 2 :
            amat=np.tile(aval,[14,14,1])*svec[li]*lyrBL[li]
        attnmats.append(amat) 
    return attnmats
        
        
        
        
        

In [16]:
def make_amats(oind,svec): #tuning-based attention

    attnmats=[];
    with open(TCpath+'/featvecs20_train35_c.txt', "rb") as fp:  b = pickle.load(fp,encoding='latin1')
    for li in range(2):
        fv=b[li]; fmvals=np.squeeze(fv[oind,:]) #fmvals=np.random.permutation(fmvals)# shuffle vals across feat maps
        aval=np.expand_dims(np.expand_dims(fmvals,axis=0),axis=0); #ori, fm
        aval[aval==np.inf]=0; aval[aval==-np.inf]=0; aval=np.nan_to_num(aval)
        if bd==0:
            aval[aval<0]=0
        if attype==1:
            amat=np.ones((224,224,64))+np.tile(aval,[224,224,1])*svec[li]; amat[amat<0]=0
        elif attype==2:
            amat=np.tile(aval,[224,224,1])*svec[li]*lyrBL[li]; 
        attnmats.append(amat)
        #print amat
    for li in range(2,4):
        fv=b[li]; fmvals=np.squeeze(fv[oind,:]) #fmvals=np.random.permutation(fmvals)# shuffle vals across feat maps
        aval=np.expand_dims(np.expand_dims(fmvals,axis=0),axis=0); #ori, fm
        aval[aval==np.inf]=0; aval[aval==-np.inf]=0; aval=np.nan_to_num(aval)
        if bd==0:
            aval[aval<0]=0
        if attype==1:
            amat=np.ones((112,112,128))+np.tile(aval,[112,112,1])*svec[li]; amat[amat<0]=0
        elif attype==2:
            amat=np.tile(aval,[112,112,1])*svec[li]*lyrBL[li]; 
        attnmats.append(amat)
        
    for li in range(4,7):
        fv=b[li]; fmvals=np.squeeze(fv[oind,:]) #fmvals=np.random.permutation(fmvals)# shuffle vals across feat maps
        aval=np.expand_dims(np.expand_dims(fmvals,axis=0),axis=0); #ori, fm
        aval[aval==np.inf]=0; aval[aval==-np.inf]=0; aval=np.nan_to_num(aval)
        if bd==0:
            aval[aval<0]=0
        if attype==1:
            amat=np.ones((56,56,256))+np.tile(aval,[56,56,1])*svec[li]; amat[amat<0]=0
        elif attype==2:
            amat=np.tile(aval,[56,56,1])*svec[li]*lyrBL[li]; 
        attnmats.append(amat)
        
    for li in range(7,10):
        fv=b[li]; fmvals=np.squeeze(fv[oind,:]) #fmvals=np.random.permutation(fmvals)# shuffle vals across feat maps
        aval=np.expand_dims(np.expand_dims(fmvals,axis=0),axis=0); #ori, fm
        aval[aval==np.inf]=0; aval[aval==-np.inf]=0; aval=np.nan_to_num(aval)
        if bd==0:
            aval[aval<0]=0
        if attype==1:
            amat=np.ones((28,28,512))+np.tile(aval,[28,28,1])*svec[li]; amat[amat<0]=0
        elif attype==2:
            amat=np.tile(aval,[28,28,1])*svec[li]*lyrBL[li]; 
        attnmats.append(amat)
        
    for li in range(10,13):
        fv=b[li]; fmvals=np.squeeze(fv[oind,:]) #fmvals=np.random.permutation(fmvals)# shuffle vals across feat maps
        aval=np.expand_dims(np.expand_dims(fmvals,axis=0),axis=0); #ori, fm
        aval[aval==np.inf]=0; aval[aval==-np.inf]=0; aval=np.nan_to_num(aval)
        if bd==0:
            aval[aval<0]=0
        if attype==1:
            amat=np.ones((14,14,512))+np.tile(aval,[14,14,1])*svec[li]; amat[amat<0]=0
        elif attype==2:
            amat=np.tile(aval,[14,14,1])*svec[li]*lyrBL[li]; 
        attnmats.append(amat)
    #print amat
    return attnmats

VGG16 structure

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class VGG16(nn.Module):

    def __init__(self,weights,avalue,binary_weight):
        super(VGG16, self).__init__()
        self.weights = weights
        self.avalue = avalue
        self.binary_weight = binary_weight
        #conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3),stride=1, padding=1, padding_mode='zeros', bias=True)
        #conv1_1.weight = nn.parameter(torch.from_numpy(self.weights['conv1_1_W']).permute(2,3,0,1))
        #conv1_1.bias = torch.nn.parameter(torch.from_numpy(weights['conv1_1_b']))
        #self.conv1_1 = conv1_1
        # 3 * 224 * 224
        self.conv1_1 = nn.Conv2d(3, 64, 3,bias=True,padding=1)  # 64 * 222 * 222
        self.conv1_1.weight = nn.Parameter(torch.from_numpy(self.weights['conv1_1_W'].astype(np.double)).permute(3,2,0,1))
        self.conv1_1.weight.require_grads = False
        self.conv1_1.bias = nn.Parameter(torch.from_numpy(self.weights['conv1_1_b'].astype(np.double)))
        self.conv1_1.bias.require_grads = False
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=(1, 1),bias=True)  # 64 * 222* 222
        self.conv1_2.weight = nn.Parameter(torch.from_numpy(self.weights['conv1_2_W'].astype(np.double)).permute(3,2,0,1))
        self.conv1_2.bias = nn.Parameter(torch.from_numpy(self.weights['conv1_2_b'].astype(np.double)))
        self.conv1_2.weight.require_grads = False
        self.conv1_2.bias.require_grads = False
        self.maxpool1 = nn.MaxPool2d((2, 2))  # pooling 64 * 112 * 112

        self.conv2_1 = nn.Conv2d(64, 128, 3,bias=True,padding=1)  # 128 * 110 * 110
        self.conv2_1.weight = nn.Parameter(torch.from_numpy(self.weights['conv2_1_W'].astype(np.double)).permute(3,2,0,1))
        self.conv2_1.bias = nn.Parameter(torch.from_numpy(self.weights['conv2_1_b'].astype(np.double)))
        self.conv2_1.weight.require_grads = False
        self.conv2_1.bias.require_grads = False
        
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=(1, 1),bias=True)  # 128 * 110 * 110
        self.conv2_2.weight = nn.Parameter(torch.from_numpy(self.weights['conv2_2_W'].astype(np.double)).permute(3,2,0,1))
        self.conv2_2.bias = nn.Parameter(torch.from_numpy(self.weights['conv2_2_b'].astype(np.double)))
        self.conv2_2.weight.require_grads = False
        self.conv2_2.bias.require_grads = False
        
        self.maxpool2 = nn.MaxPool2d((2, 2))  # pooling 128 * 56 * 56

        self.conv3_1 = nn.Conv2d(128, 256, 3,bias=True,padding=1)  # 256 * 54 * 54
        self.conv3_1.weight = nn.Parameter(torch.from_numpy(self.weights['conv3_1_W'].astype(np.double)).permute(3,2,0,1))
        self.conv3_1.bias = nn.Parameter(torch.from_numpy(self.weights['conv3_1_b'].astype(np.double)))
        self.conv3_1.weight.require_grads = False
        self.conv3_1.bias.require_grads = False       
        
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=(1, 1),bias=True)  # 256 * 54 * 54
        self.conv3_2.weight = nn.Parameter(torch.from_numpy(self.weights['conv3_2_W'].astype(np.double)).permute(3,2,0,1))
        self.conv3_2.bias = nn.Parameter(torch.from_numpy(self.weights['conv3_2_b'].astype(np.double)))
        self.conv3_2.weight.require_grads = False
        self.conv3_2.bias.require_grads = False          
        
        self.conv3_3 = nn.Conv2d(256, 256, 3, padding=(1, 1),bias=True)  # 256 * 54 * 54
        self.conv3_3.weight = nn.Parameter(torch.from_numpy(self.weights['conv3_3_W'].astype(np.double)).permute(3,2,0,1))
        self.conv3_3.bias = nn.Parameter(torch.from_numpy(self.weights['conv3_3_b'].astype(np.double)))
        self.conv3_3.weight.require_grads = False
        self.conv3_3.bias.require_grads = False         
        
        self.maxpool3 = nn.MaxPool2d((2, 2))  # pooling 256 * 28 * 28

        self.conv4_1 = nn.Conv2d(256, 512, 3,bias=True,padding=1)  # 512 * 26 * 26
        self.conv4_1.weight = nn.Parameter(torch.from_numpy(self.weights['conv4_1_W'].astype(np.double)).permute(3,2,0,1))
        self.conv4_1.bias = nn.Parameter(torch.from_numpy(self.weights['conv4_1_b'].astype(np.double)))
        self.conv4_1.weight.require_grads = False
        self.conv4_1.bias.require_grads = False        
        
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=(1, 1),bias=True)  # 512 * 26 * 26
        self.conv4_2.weight = nn.Parameter(torch.from_numpy(self.weights['conv4_2_W'].astype(np.double)).permute(3,2,0,1))
        self.conv4_2.bias = nn.Parameter(torch.from_numpy(self.weights['conv4_2_b'].astype(np.double)))
        self.conv4_2.weight.require_grads = False
        self.conv4_2.bias.require_grads = False         
        
        self.conv4_3 = nn.Conv2d(512, 512, 3, padding=(1, 1),bias=True)  # 512 * 26 * 26
        self.conv4_3.weight = nn.Parameter(torch.from_numpy(self.weights['conv4_3_W'].astype(np.double)).permute(3,2,0,1))
        self.conv4_3.bias = nn.Parameter(torch.from_numpy(self.weights['conv4_3_b'].astype(np.double)))
        self.conv4_3.weight.require_grads = False
        self.conv4_3.bias.require_grads = False         
        
        self.maxpool4 = nn.MaxPool2d((2, 2))  # pooling 512 * 14 * 14

        self.conv5_1 = nn.Conv2d(512, 512, 3,bias=True,padding=1)  # 512 * 12 * 12
        self.conv5_1.weight = nn.Parameter(torch.from_numpy(self.weights['conv5_1_W'].astype(np.double)).permute(3,2,0,1))
        self.conv5_1.bias = nn.Parameter(torch.from_numpy(self.weights['conv5_1_b'].astype(np.double)))
        self.conv5_1.weight.require_grads = False
        self.conv5_1.bias.require_grads = False        
        
        self.conv5_2 = nn.Conv2d(512, 512, 3, padding=(1, 1),bias=True)  # 512 * 12 * 12
        self.conv5_2.weight = nn.Parameter(torch.from_numpy(self.weights['conv5_2_W'].astype(np.double)).permute(3,2,0,1))
        self.conv5_2.bias = nn.Parameter(torch.from_numpy(self.weights['conv5_2_b'].astype(np.double)))
        self.conv5_2.weight.require_grads = False
        self.conv5_2.bias.require_grads = False            
        
        self.conv5_3 = nn.Conv2d(512, 512, 3, padding=(1, 1),bias=True)  # 512 * 12 * 12
        self.conv5_3.weight = nn.Parameter(torch.from_numpy(self.weights['conv5_3_W'].astype(np.double)).permute(3,2,0,1))
        self.conv5_3.bias = nn.Parameter(torch.from_numpy(self.weights['conv5_3_b'].astype(np.double)))
        self.conv5_3.weight.require_grads = False
        self.conv5_3.bias.require_grads = False          
        
        self.maxpool5 = nn.MaxPool2d((2, 2))  # pooling 512 * 7 * 7
        

        # view

        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc1.weight = nn.Parameter(torch.from_numpy(self.weights['fc6_W'].astype(np.double)).permute(1,0))
        self.fc1.bias = nn.Parameter(torch.from_numpy(self.weights['fc6_b'].astype(np.double)))
        self.fc1.weight.require_grads = False
        self.fc1.bias.require_grads = False
        
        self.fc2 = nn.Linear(4096, 4096)
        self.fc2.weight = nn.Parameter(torch.from_numpy(self.weights['fc7_W'].astype(np.double)).permute(1,0))
        self.fc2.bias = nn.Parameter(torch.from_numpy(self.weights['fc7_b'].astype(np.double)))
        self.fc2.weight.require_grads = False
        self.fc2.bias.require_grads = False        
        
        self.fc3 = nn.Linear(4096, 1)
        self.fc3.weight = nn.Parameter(torch.from_numpy(self.binary_weight.get_tensor('fc3').astype(np.double)).permute(1,0))
        self.fc3.bias = nn.Parameter(torch.from_numpy(self.binary_weight.get_tensor('fcb3').astype(np.double)))
        self.fc3.weight.require_grads = False
        self.fc3.bias.require_grads = False  
 #       self.fc3.weight = nn.Parameter(torch.from_numpy(weights['fc8_W'].astype(np.double)).permute(1,0)[:2,:])
 #       self.fc3.bias = nn.Parameter(torch.from_numpy(weights['fc8_b'].astype(np.double))[:2])
       

    def forward(self, x):
        # x.size(0)即为batch_size
        in_size = x.size(0)

        out = self.conv1_1(x)  # 222
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[0]).permute(2,0,1))
        out = self.conv1_2(out)  # 222
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[1]).permute(2,0,1))
        out = self.maxpool1(out)  # 112

        out = self.conv2_1(out)  # 110
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[2]).permute(2,0,1))
        out = self.conv2_2(out)  # 110
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[3]).permute(2,0,1))
        out = self.maxpool2(out)  # 56

        out = self.conv3_1(out)  # 54
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[4]).permute(2,0,1))
        out = self.conv3_2(out)  # 54
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[5]).permute(2,0,1))
        out = self.conv3_3(out)  # 54
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[6]).permute(2,0,1))
        out = self.maxpool3(out)  # 28

        out = self.conv4_1(out)  # 26
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[7]).permute(2,0,1))
        out = self.conv4_2(out)  # 26
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[8]).permute(2,0,1))
        out = self.conv4_3(out)  # 26
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[9]).permute(2,0,1))
        out = self.maxpool4(out)  # 14

        out = self.conv5_1(out)  # 12
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[10]).permute(2,0,1))
        out = self.conv5_2(out)  # 12
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[11]).permute(2,0,1))
        out = self.conv5_3(out)  # 12
        out = F.relu(out)
        out = torch.mul(out,torch.from_numpy(self.avalue[12]).permute(2,0,1))
        out = self.maxpool5(out)  # 7
        
        out = torch.transpose(out, 1, 3)
        out = torch.transpose(out, 1, 2)
        # 展平
        out = out.contiguous().view(75, -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
 #       out = torch.round(nn.Sigmoid(out))
 #       out = F.softmax(out, dim=1)
        return out

Preprocess image

In [18]:
def preprocess_im(image):
    image = torch.from_numpy(image)
    mean = torch.tensor([123.68, 116.779, 103.939]).reshape([1,1,1,3])
    images = image -mean
    return images

In [19]:
descat=cat 
if imtype==1:   
    descatpics=np.load(impath+'/merg5_c'+str(descat)+'.npz')['arr_0']
elif imtype==2:
    descatpics=np.load(impath+'/arr5_c'+str(descat)+'.npz')['arr_0']
elif imtype==3:
    descatpics=np.load(impath+'/cats20_test15_c.npy')

tp_batch=np.zeros((bsize,224,224,3)); tplabs=np.ones((bsize,1))
tn_batch=np.zeros((bsize,224,224,3)); tnlabs=np.zeros((bsize,1))
TPscore=np.zeros((len(astrgs))); TNscore=np.zeros((len(astrgs))); #FN=np.zeros((len(str_vec),xval)); TN=np.zeros((len(str_vec),xval))

if rec_activity:
    tp_resps1=np.ones((2,bsize,64,len(astrgs))); tp_resps2=np.ones((2,bsize,128,len(astrgs)));  tp_resps3=np.ones((3,bsize,256,len(astrgs)))
    tp_resps4=np.ones((3,bsize,512,len(astrgs))); tp_resps5=np.ones((3,bsize,512,len(astrgs)))
    tn_resps1=np.ones((2,bsize,64,len(astrgs))); tn_resps2=np.ones((2,bsize,128,len(astrgs)));  tn_resps3=np.ones((3,bsize,256,len(astrgs)))
    tn_resps4=np.ones((3,bsize,512,len(astrgs))); tn_resps5=np.ones((3,bsize,512,len(astrgs)))

np.random.seed(10) #so each layer has same images etc
if imtype==3:
    tp_batch=descatpics[descat]
else:
    for pii in range(15):
        tp_batch[pii*imperim:(pii+1)*imperim,:,:,:]=descatpics[pii,np.random.choice(19*5,imperim),:,:,:]
del descatpics

othercs=np.arange(Ncats); othercs=othercs[othercs!=descat]; cii=0
if imtype==3:
    othercs=np.random.choice(othercs,15)
else:
    imspercats=np.ones((Ncats-1))*impercat; imspercats[np.random.choice(Ncats-1,leftov)]+=1
# print Ncats, type(descat), othercs
for ci in range(len(othercs)):
    if imtype==1:
        ocatpicsF=np.load(impath+'/merg5_c'+str(othercs[ci])+'.npz')
        ocatpics=ocatpicsF['arr_0']; ocatlabs=ocatpicsF['arr_1']; noncind=np.where(ocatlabs[0,:]!=descat)
        del ocatpicsF
        tn_batch[cii:cii+int(imspercats[ci]),:,:,:]=ocatpics[np.random.choice(15,int(imspercats[ci])),np.random.choice(noncind[0],int(imspercats[ci])),:,:,:]
        del ocatpics
        cii+=int(imspercats[ci])
    elif imtype==2:
        ocatpicsF=np.load(impath+'/arr5_c'+str(othercs[ci])+'.npz')
        ocatpics=ocatpicsF['arr_0']; ocatlabs=ocatpicsF['arr_1']; 
        del ocatpicsF
        for pici in range(int(imspercats[ci])):            
            pic=np.random.choice(15); binocat=np.sum(np.squeeze(ocatlabs[pic,:,:])==descat,axis=1)
            print(cii)
            print(binocat)
            noncind=np.where(binocat==0) #need to check that all 3 others arent descat
            tn_batch[cii+pici:cii+pici+1,:,:,:]=ocatpics[pic,np.random.choice(noncind[0]),:,:,:]
            cii+=imspercats[ci]
    elif imtype==3:
        ocatpicsF=descatpics[othercs[ci]]  
        tn_batch[ci,:,:,:]=ocatpicsF[np.random.choice(15),:,:,:]
        del ocatpicsF


In [20]:
wieghtpath = '/scratch/bs4283/Data/Data'
weights = np.load(wieghtpath+'/vgg16_weights.npz')

In [21]:
cat

19

In [22]:
a_n = preprocess_im(tn_batch)
a_p = preprocess_im(tp_batch)
a_n = a_n.permute(0,3,1,2)
a_p = a_p.permute(0,3,1,2)
poi_res = np.zeros([2,12])
neg_res = np.zeros([2,12])
kk = 0
lyr=kk
laybins=np.zeros((13))
if lyr>12:   
    laybins=np.ones((13)); astrgs=astrgs/10.0
else:
    laybins[lyr]=1
ai =  1
astrg = 0
if appwith == 'TCs':    
    attnmats=make_amats(descat,laybins*astrg) #descat = cat, cat is 0-19 for which object category to attend to and readout for
    attnmats = attnmats[(int(len(attnmats)/13)-1)*13:len(attnmats)+1]
elif appwith=='GRADs':
    attnmats=make_gamats(descat,laybins*astrg)       
model = VGG16(weights=weights,avalue=attnmats,binary_weight = reader)
model.eval()
tp_score = model(a_p)
tn_score = model(a_n)
tp_score = tp_score.detach().numpy()
tn_score = tn_score.detach().numpy()

In [23]:
prr = np.sum(tp_score>0)/75
nrr = np.sum(tn_score<0)/75

In [26]:
poi_res = np.ones([2,12]) * prr
neg_res = np.ones([2,12])
for kk in range(12):
    lyr=kk
    laybins=np.zeros((13))
    if lyr>12:   
        laybins=np.ones((13)); astrgs=astrgs/10.0
    else:
        laybins[lyr]=1
    ai =  1
    astrg = 0.5
    if appwith == 'TCs':    
        attnmats=make_amats(descat,laybins*astrg) #descat = cat, cat is 0-19 for which object category to attend to and readout for
        attnmats = attnmats[(int(len(attnmats)/13)-1)*13:len(attnmats)+1]
    elif appwith=='GRADs':
        attnmats=make_gamats(descat,laybins*astrg)       
    model = VGG16(weights=weights,avalue=attnmats,binary_weight = reader)
    model.eval()
    tp_score = model(a_p)
    tn_score = model(a_n)
    tp_score = tp_score.detach().numpy()
    tn_score = tn_score.detach().numpy()
    poi_res[ai,kk] = np.sum(tp_score>0)/75
    neg_res[ai,kk] = np.sum(tn_score<0)/75

In [ ]:
a_n = preprocess_im(tn_batch)
a_p = preprocess_im(tp_batch)
a_n = a_n.permute(0,3,1,2)
a_p = a_p.permute(0,3,1,2)
poi_res = np.zeros([2,12])
neg_res = np.zeros([2,12])
for kk in range(12):
    ai = -1 
    lyr=kk
    laybins=np.zeros((13))
    if lyr>12:   
        laybins=np.ones((13)); astrgs=astrgs/10.0
    else:
        laybins[lyr]=1
    for astrg in astrgs:
        ai =ai+1 
        if appwith == 'TCs':
            attnmats=make_amats(descat,laybins*astrg) #descat = cat, cat is 0-19 for which object category to attend to and readout for
            attnmats = attnmats[(int(len(attnmats)/13)-1)*13:len(attnmats)+1]
        elif appwith=='GRADs':
            attnmats=make_gamats(descat,laybins*astrg)       
        model = VGG16(weights=weights,avalue=attnmats,binary_weight = reader)
        model.eval()
        tp_score = model(a_p)
        tn_score = model(a_n)
        tp_score = tp_score.detach().numpy()
        tn_score = tn_score.detach().numpy()
        poi_res[ai,kk] = np.sum(tp_score>0)/75
        neg_res[ai,kk] = np.sum(tn_score<0)/75

In [ ]:
ai =ai+1 
if appwith == 'TCs':    
    attnmats=make_amats(descat,laybins*astrg) #descat = cat, cat is 0-19 for which object category to attend to and readout for
    attnmats = attnmats[(int(len(attnmats)/13)-1)*13:len(attnmats)+1]
elif appwith=='GRADs':
    attnmats=make_gamats(descat,laybins*astrg)       
model = VGG16(weights=weights,avalue=attnmats,binary_weight = reader)
model.eval()
tp_score = model(a_p)
tn_score = model(a_n)
tp_score = tp_score.detach().numpy()
tn_score = tn_score.detach().numpy()
poi_res[ai,kk] = np.sum(tp_score>0)/75
neg_res[ai,kk] = np.sum(tn_score<0)/75

In [29]:
np.save('/scratch/bs4283/Data/result/cat19_poi.npy',poi_res)
np.save('/scratch/bs4283/Data/result/cat19_neg.npy',neg_res)